In [1]:
!pip install psycopg2-binary sqlalchemy pandas

  Using cached psycopg2_binary-2.9.11-cp313-cp313-macosx_10_13_x86_64.whl.metadata (4.9 kB)
  Using cached sqlalchemy-2.0.45-py3-none-any.whl.metadata (9.5 kB)
  Using cached pandas-2.3.3-cp313-cp313-macosx_10_13_x86_64.whl.metadata (91 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached psycopg2_binary-2.9.11-cp313-cp313-macosx_10_13_x86_64.whl (3.8 MB)
Using cached sqlalchemy-2.0.45-py3-none-any.whl (1.9 MB)
Using cached pandas-2.3.3-cp313-cp313-macosx_10_13_x86_64.whl (11.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 4.5 MB/s  0:00:01 eta 0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
Using cached tzdata-2025.3-py2.py3-none-any.whl (348 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [pandas]2m7/8 [pandas]emy]nary]s]

Importing Necessary Libraries

In [2]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

Data Loading

In [3]:
# load data into pandas DataFrame
f4f_df = pd.read_csv('raw_data/flour4four_orders_oct2025.csv')
f4f_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          10000 non-null  object 
 1   order_date        9372 non-null   object 
 2   delivery_date     10000 non-null  object 
 3   business_id       10000 non-null  object 
 4   business_name     9398 non-null   object 
 5   business_type     9013 non-null   object 
 6   business_address  9022 non-null   object 
 7   contact_name      10000 non-null  object 
 8   contact_phone     10000 non-null  int64  
 9   flour_type        9358 non-null   object 
 10  quantity_bags     10000 non-null  int64  
 11  price_per_bag     9364 non-null   float64
 12  total_amount      10000 non-null  int64  
 13  payment_method    10000 non-null  object 
 14  order_status      10000 non-null  object 
 15  rider_name        10000 non-null  object 
 16  rider_phone       10000 non-null  int64  

Data Cleaning

In [ ]:
# missing values handling
f4f_df['order_date'] = f4f_df['order_date'].fillna(f4f_df['delivery_date'])
f4f_df['business_name'] = f4f_df['business_name'].fillna('unknown')
f4f_df['business_type'] = f4f_df['business_type'].fillna('unknown')
f4f_df['business_address'] = f4f_df['business_address'].fillna('unknown')
f4f_df['flour_type'] = f4f_df['flour_type'].fillna('unknown')
f4f_df['price_per_bag'] = f4f_df['price_per_bag'].fillna(
    f4f_df['price_per_bag'].median())

f4f_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          10000 non-null  object 
 1   order_date        10000 non-null  object 
 2   delivery_date     10000 non-null  object 
 3   business_id       10000 non-null  object 
 4   business_name     10000 non-null  object 
 5   business_type     10000 non-null  object 
 6   business_address  10000 non-null  object 
 7   contact_name      10000 non-null  object 
 8   contact_phone     10000 non-null  int64  
 9   flour_type        10000 non-null  object 
 10  quantity_bags     10000 non-null  int64  
 11  price_per_bag     10000 non-null  float64
 12  total_amount      10000 non-null  int64  
 13  payment_method    10000 non-null  object 
 14  order_status      10000 non-null  object 
 15  rider_name        10000 non-null  object 
 16  rider_phone       10000 non-null  int64  

Data types conversion

In [ ]:
# convert data types
f4f_df['order_date'] = pd.to_datetime(f4f_df['order_date'])
f4f_df['delivery_date'] = pd.to_datetime(f4f_df['delivery_date'])

f4f_df['contact_phone'] = f4f_df['contact_phone'].astype(str)
f4f_df['rider_phone'] = f4f_df['rider_phone'].astype(str)

f4f_df['quantity_bags'] = f4f_df['quantity_bags'].astype(int)
f4f_df['price_per_bag'] = f4f_df['price_per_bag'].astype(float)
f4f_df['total_amount'] = f4f_df['total_amount'].astype(float)

f4f_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          10000 non-null  object        
 1   order_date        10000 non-null  datetime64[ns]
 2   delivery_date     10000 non-null  datetime64[ns]
 3   business_id       10000 non-null  object        
 4   business_name     10000 non-null  object        
 5   business_type     10000 non-null  object        
 6   business_address  10000 non-null  object        
 7   contact_name      10000 non-null  object        
 8   contact_phone     10000 non-null  object        
 9   flour_type        10000 non-null  object        
 10  quantity_bags     10000 non-null  int64         
 11  price_per_bag     10000 non-null  float64       
 12  total_amount      10000 non-null  float64       
 13  payment_method    10000 non-null  object        
 14  order_status      10000

In [10]:
f4f_df.columns

Index(['order_id', 'order_date', 'delivery_date', 'business_id',
       'business_name', 'business_type', 'business_address', 'contact_name',
       'contact_phone', 'flour_type', 'quantity_bags', 'price_per_bag',
       'total_amount', 'payment_method', 'order_status', 'rider_name',
       'rider_phone'],
      dtype='object')

Data Transformation

In [ ]:

business_df = f4f_df[[
    'business_id',
    'business_name',
    'business_type',
    'business_address',
    'contact_name',
    'contact_phone'
]].drop_duplicates(subset='business_id').reset_index(drop=True)
business_df.head()

,business_id,business_name,business_type,business_address,contact_name,contact_phone
0,BIZ-1018,"Adams, Zuniga and Wong",Restaurant,"Herbert Macaulay Way, Abuja",Elimu Agbaje,8017507864
1,BIZ-1006,Blake and Sons,Bakery,"Ahmadu Bello Way, Abuja",Bolanle Kalumba,8055667651
2,BIZ-1052,Chapman and Sons,Cafe,"Ahmadu Bello Way, Abuja",Hassan Nyoni,8083863413
3,BIZ-1035,Rodriguez-Graham,Restaurant,"Herbert Macaulay Way, Abuja",Mandela Onyango,8075228535
4,BIZ-1039,"Romero, Gonzalez and Brooks",unknown,"Garki Area 1, Abuja",Mojisola Seko,8060119651


In [ ]:
rider_df = f4f_df[[
    'rider_name',
    'rider_phone'
]].drop_duplicates().reset_index(drop=True)

rider_df['rider_id'] = range(1, len(rider_df) + 1)
rider_df = rider_df[['rider_id', 'rider_name', 'rider_phone']]
rider_df.head()

,rider_id,rider_name,rider_phone
0,1,Aisha Bello,8089864260
1,2,Tunde Oladipo,8019121552
2,3,Emeka John,8019196777
3,4,Grace Onyema,8041568532


In [18]:
flour_df = f4f_df[['flour_type']].drop_duplicates().reset_index(drop=True)
flour_df['flour_type_id'] = range(1, len(flour_df) + 1)
flour_df = flour_df[['flour_type_id', 'flour_type']]
flour_df.head()

,flour_type_id,flour_type
0,1,unknown
1,2,Bread Flour
2,3,Pastry Flour
3,4,All-purpose
4,5,Whole Wheat


In [19]:
# fact_orders table
fact_order_df = f4f_df.copy()

fact_orders_df = f4f_df.merge(flour_df, on='flour_type', how='left') \
    .merge(rider_df, on=['rider_name', 'rider_phone'], how='left') \
        [['order_id', 'order_date', 'delivery_date', 'business_id', 'flour_type_id', 'rider_id', 'quantity_bags', 'price_per_bag', 'total_amount', 'payment_method', 'order_status']]

fact_orders_df.head()

,order_id,order_date,delivery_date,business_id,flour_type_id,rider_id,quantity_bags,price_per_bag,total_amount,payment_method,order_status
0,ORD-214576,2025-10-25,2025-10-25,BIZ-1018,1,1,26,9500.0,247000.0,POS,Delivered
1,ORD-299448,2025-10-08,2025-10-08,BIZ-1006,2,2,27,10000.0,270000.0,POS,Cancelled
2,ORD-246991,2025-10-17,2025-10-17,BIZ-1052,3,2,21,9800.0,205800.0,Bank Transfer,Cancelled
3,ORD-392075,2025-10-13,2025-10-13,BIZ-1035,4,2,20,10500.0,210000.0,Bank Transfer,Pending
4,ORD-179046,2025-10-14,2025-10-14,BIZ-1039,2,3,40,10000.0,380000.0,POS,Pending


DATA LOADING

In [20]:
# Establish database connection on PostgreSQL
db_username = 'postgres'
db_password = '#Adejoke8509'
db_host = 'localhost'
db_port = '5432'
db_name = 'flour_for_four_db'

In [21]:
# Define connection to psycopg2
def get_db_connection():
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_username,
        password=db_password,
        host=db_host,
        port=db_port
    )
    return conn

conn = get_db_connection()

In [23]:
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_tables_query = """
                CREATE SCHEMA IF NOT EXISTS flour_for_four;

                DROP TABLE IF EXISTS flour_for_four.fact_orders_df CASCADE;
                DROP TABLE IF EXISTS flour_for_four.business_df CASCADE;
                DROP TABLE IF EXISTS flour_for_four.flour_df CASCADE;
                DROP TABLE IF EXISTS flour_for_four.rider_df CASCADE;

                CREATE TABLE flour_for_four.business_df (
                    business_id       VARCHAR PRIMARY KEY,
                    business_name     VARCHAR NOT NULL,
                    business_type     VARCHAR,
                    business_address  VARCHAR,
                    contact_name      VARCHAR NOT NULL,
                    contact_phone     VARCHAR NOT NULL
                );

                CREATE TABLE flour_for_four.flour_df (
                    flour_type_id SERIAL PRIMARY KEY,
                    flour_type VARCHAR
                );

                CREATE TABLE flour_for_four.rider_df (
                    rider_id SERIAL PRIMARY KEY,
                    rider_name VARCHAR NOT NULL,
                    rider_phone VARCHAR NOT NULL
                );

                CREATE TABLE flour_for_four.fact_orders_df (
                    order_id       VARCHAR PRIMARY KEY,
                    order_date     DATE NOT NULL,
                    delivery_date  DATE NOT NULL,
                    business_id    VARCHAR NOT NULL REFERENCES flour_for_four.business_df(business_id),
                    rider_id    INT NOT NULL REFERENCES flour_for_four.rider_df(rider_id),
                    flour_type_id     INT NOT NULL REFERENCES flour_for_four.flour_df(flour_type_id),
                    quantity_bags  INT NOT NULL,
                    price_per_bag   NUMERIC NOT NULL,
                    total_amount   NUMERIC NOT NULL,
                    payment_method VARCHAR NOT NULL,
                    order_status   VARCHAR NOT NULL
                );
    """
    cursor.execute(create_tables_query)
    conn.commit()
    cursor.close()
    conn.close()
    print("Tables created successfully.")
    
    
create_tables()

Tables created successfully.


In [24]:
# Load data into the database tables

engine = create_engine(f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

business_df.to_sql('business_df', engine, schema='flour_for_four', if_exists='append', index=False)

flour_df.to_sql('flour_df', engine, schema='flour_for_four', if_exists='append', index=False)

rider_df.to_sql('rider_df', engine, schema='flour_for_four', if_exists='append', index=False)

fact_orders_df.to_sql('fact_orders_df', engine, schema='flour_for_four', if_exists='append', index=False)

1000